In [204]:
import pandas as pd
import ast
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

In [184]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Importing the data

In [128]:
user_attributes=pd.read_excel("Satish Sahu_ Data Science test.xlsx",sheet_name="Users Attributes")
email_data=pd.read_excel("Satish Sahu_ Data Science test.xlsx",sheet_name="Email Data")
content_usage=pd.read_excel("Satish Sahu_ Data Science test.xlsx",sheet_name="Content Usage")

/Users/satishkumarsahu/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/satishkumarsahu/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


## Restructuring data

In [129]:
# Load the email data
email_data['category']=email_data['category'].str.replace('\[|\]|\"','')
email_data['category']=email_data['category'].str.lower()

/var/folders/r9/kplwdy9n4d90cxzq9zls2dv40000gp/T/ipykernel_4320/1201657326.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  email_data['category']=email_data['category'].str.replace('\[|\]|\"','')


In [130]:
email_data.category.value_counts()

live_class_one_hour_reminder                  6822
weeklyreport                                  3605
ws_reminder_02a                               2210
live_class_weekly_reminder                    1854
ws_reminder_01a                               1778
                                              ... 
ptl_teacherinvitesparent_reminder                1
unconfirmed email account deletion warning       1
web_yearly_purchase_384day                       1
socialprivacyemail                               1
app_yearly_purchase_320day                       1
Name: category, Length: 73, dtype: int64

In [131]:


# Pivot the data to create new columns for each email category
email_data = pd.pivot_table(email_data, index="Subscription ID", columns="category", aggfunc="size", fill_value=0)

# Rename the new columns to indicate the count of emails received
email_data.columns = ["email_category_" + str(col) + "_count" for col in email_data.columns]

In [132]:
email_data

,email_category_activation_count,email_category_app_monthly_purchase_15day_count,email_category_app_yearly_purchase_135day_count,email_category_app_yearly_purchase_200day_count,email_category_app_yearly_purchase_260day_count,email_category_app_yearly_purchase_320day_count,email_category_app_yearly_purchase_75day_count,email_category_apps_day3_pa_1_count,email_category_apps_day3_pa_2_count,email_category_cancellation_confirmation_count,...,email_category_ws_reminder_01a_count,email_category_ws_reminder_01b_count,email_category_ws_reminder_01c_count,email_category_ws_reminder_02a_count,email_category_ws_reminder_02b_count,email_category_ws_reminder_02c_count,email_category_ws_reminder_02d_count,email_category_ws_reminder_02e_count,email_category_ws_reminder_03a_count,email_category_ws_reminder_03b_count
Subscription ID,,,,,,,,,,,,,,,,,,,,,
002B07927A974E0C80A278EACC7C4B4C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0033B484E5014852BD4FBE99487A2CB5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
004CF49B88AD4203BF18EBC366695648,0,0,0,0,0,0,0,0,0,1,...,1,0,0,1,1,0,0,0,0,0
00517CC4ACFF4F16A19C5B5711E25463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
005989B404624D31AFBA649E2B18DFD7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FF379C8F8AEC4E99B5B3DED91A8F66A6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
FF52DE445E1C4B3296B289C29EE2BAE0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,2,0,0,2,2,0,0
FF644E59F64C42AE852C8F5069733C22,0,0,0,0,0,0,0,0,0,1,...,1,0,0,2,1,1,1,1,0,0


In [133]:
email_data.reset_index(inplace=True)

In [134]:
email_data

,Subscription ID,email_category_activation_count,email_category_app_monthly_purchase_15day_count,email_category_app_yearly_purchase_135day_count,email_category_app_yearly_purchase_200day_count,email_category_app_yearly_purchase_260day_count,email_category_app_yearly_purchase_320day_count,email_category_app_yearly_purchase_75day_count,email_category_apps_day3_pa_1_count,email_category_apps_day3_pa_2_count,...,email_category_ws_reminder_01a_count,email_category_ws_reminder_01b_count,email_category_ws_reminder_01c_count,email_category_ws_reminder_02a_count,email_category_ws_reminder_02b_count,email_category_ws_reminder_02c_count,email_category_ws_reminder_02d_count,email_category_ws_reminder_02e_count,email_category_ws_reminder_03a_count,email_category_ws_reminder_03b_count
0,002B07927A974E0C80A278EACC7C4B4C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0033B484E5014852BD4FBE99487A2CB5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,004CF49B88AD4203BF18EBC366695648,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
3,00517CC4ACFF4F16A19C5B5711E25463,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,005989B404624D31AFBA649E2B18DFD7,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133,FF379C8F8AEC4E99B5B3DED91A8F66A6,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
3134,FF52DE445E1C4B3296B289C29EE2BAE0,0,0,0,0,0,0,0,0,0,...,2,0,0,2,0,0,2,2,0,0
3135,FF644E59F64C42AE852C8F5069733C22,0,0,0,0,0,0,0,0,0,...,1,0,0,2,1,1,1,1,0,0
3136,FF761B75DB5D4869A8E264FB35E168F0,0,0,0,0,0,0,0,0,0,...,3,0,0,2,3,0,0,0,0,0


In [135]:
email_data.describe()

,email_category_activation_count,email_category_app_monthly_purchase_15day_count,email_category_app_yearly_purchase_135day_count,email_category_app_yearly_purchase_200day_count,email_category_app_yearly_purchase_260day_count,email_category_app_yearly_purchase_320day_count,email_category_app_yearly_purchase_75day_count,email_category_apps_day3_pa_1_count,email_category_apps_day3_pa_2_count,email_category_cancellation_confirmation_count,...,email_category_ws_reminder_01a_count,email_category_ws_reminder_01b_count,email_category_ws_reminder_01c_count,email_category_ws_reminder_02a_count,email_category_ws_reminder_02b_count,email_category_ws_reminder_02c_count,email_category_ws_reminder_02d_count,email_category_ws_reminder_02e_count,email_category_ws_reminder_03a_count,email_category_ws_reminder_03b_count
count,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,...,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000
mean,0.125558,0.001275,0.000319,0.000319,0.000319,0.000319,0.000319,0.012110,0.007967,0.375717,...,0.566603,0.029637,0.014978,0.704270,0.553219,0.368706,0.323136,0.242192,0.026131,0.265774
std,0.442603,0.035686,0.017851,0.017851,0.017851,0.017851,0.017851,0.115073,0.088915,0.672808,...,0.880233,0.248838,0.197446,0.941983,0.813814,0.628311,0.592790,0.527815,0.203458,0.934983
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,7.000000,...,5.000000,3.000000,3.000000,5.000000,4.000000,3.000000,4.000000,2.000000,3.000000,8.000000


In [136]:
content_usage.columns

Index(['Subscription ID', 'content_category', 'played date', 'asked_count',
       'unique_playables_attempted', 'unique_playables_completed_attempted',
       'time_spent_secs'],
      dtype='object')

In [137]:
content_usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363436 entries, 0 to 363435
Data columns (total 7 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Subscription ID                       363436 non-null  object 
 1   content_category                      363293 non-null  object 
 2   played date                           363293 non-null  object 
 3   asked_count                           363262 non-null  float64
 4   unique_playables_attempted            363293 non-null  float64
 5   unique_playables_completed_attempted  363293 non-null  float64
 6   time_spent_secs                       363262 non-null  float64
dtypes: float64(4), object(3)
memory usage: 19.4+ MB


In [138]:
content_usage["played date"]=pd.to_datetime(content_usage["played date"])

In [139]:
content_usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363436 entries, 0 to 363435
Data columns (total 7 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   Subscription ID                       363436 non-null  object        
 1   content_category                      363293 non-null  object        
 2   played date                           363293 non-null  datetime64[ns]
 3   asked_count                           363262 non-null  float64       
 4   unique_playables_attempted            363293 non-null  float64       
 5   unique_playables_completed_attempted  363293 non-null  float64       
 6   time_spent_secs                       363262 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 19.4+ MB


In [140]:
# Create a pivot table to aggregate values for each content category and Subscription ID
content_usage1 = pd.pivot_table(content_usage, index="Subscription ID", columns="content_category", aggfunc={"asked_count": "sum", "unique_playables_attempted": "sum", "unique_playables_completed_attempted": "sum", "time_spent_secs": "sum"}, fill_value=0)

# Flatten the multi-level column index
content_usage1.columns = [f"{col[1]}_{col[0]}" for col in content_usage1.columns]

In [141]:
content_usage

,Subscription ID,content_category,played date,asked_count,unique_playables_attempted,unique_playables_completed_attempted,time_spent_secs
0,134100A51B984EF4B6D2095F99A6B750,ela_lp,2022-11-14 22:32:54.825588,42.0,6.0,6.0,1095.0
1,134100A51B984EF4B6D2095F99A6B750,math_lp,2022-11-14 22:48:21.646972,36.0,6.0,6.0,579.0
2,A711C54805FD44DBA36A049C2A3D9DAD,ela_lp,2022-03-18 00:57:08.676977,8.0,2.0,2.0,162.0
3,AC8E1A20F0B74FEF967E0ACD2188C58A,math_lp,2022-04-27 21:47:50.827265,15.0,2.0,2.0,310.0
4,AC8E1A20F0B74FEF967E0ACD2188C58A,math_lp,2022-04-27 21:59:07.413829,26.0,5.0,4.0,344.0
...,...,...,...,...,...,...,...
363431,8C37A223672A4DD7A5A8CD25C466FC31,math_learning_games,2022-04-13 18:31:56.792687,18.0,4.0,4.0,288.0
363432,E088D83FD8B5472BA3C368215712A5B9,NaN,NaT,NaN,NaN,NaN,NaN
363433,4CAA32D868AA4B44A6263110768DB6C1,NaN,NaT,NaN,NaN,NaN,NaN
363434,1EB4DDD477FE4955B751936CCDCAF099,NaN,NaT,NaN,NaN,NaN,NaN


In [142]:
content_usage1.reset_index(inplace=True)

In [143]:
content_usage1

,Subscription ID,activity_asked_count,books_asked_count,ela_lp_asked_count,math_learning_games_asked_count,math_lp_asked_count,mathfacts_asked_count,others_asked_count,playzone_asked_count,reading_asked_count,...,activity_unique_playables_completed_attempted,books_unique_playables_completed_attempted,ela_lp_unique_playables_completed_attempted,math_learning_games_unique_playables_completed_attempted,math_lp_unique_playables_completed_attempted,mathfacts_unique_playables_completed_attempted,others_unique_playables_completed_attempted,playzone_unique_playables_completed_attempted,reading_unique_playables_completed_attempted,spelling_unique_playables_completed_attempted
0,002B07927A974E0C80A278EACC7C4B4C,0,100,878,328,2324,0,0,2,0,...,0,12,108,52,322,0,0,0,0,4
1,0033B484E5014852BD4FBE99487A2CB5,0,0,150,332,1190,288,0,180,66,...,0,0,24,48,194,20,0,46,22,22
2,004CF49B88AD4203BF18EBC366695648,0,0,18,60,756,0,0,537,18,...,0,0,6,0,87,9,0,72,12,3
3,00517CC4ACFF4F16A19C5B5711E25463,0,0,0,150,1738,0,0,0,0,...,0,0,0,10,262,0,0,0,0,0
4,005989B404624D31AFBA649E2B18DFD7,7936,728,148,760,1328,32,0,16,0,...,12,48,8,44,156,44,0,4,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2987,FF379C8F8AEC4E99B5B3DED91A8F66A6,12,0,474,0,41,0,0,0,0,...,0,0,54,0,6,0,0,0,0,0
2988,FF52DE445E1C4B3296B289C29EE2BAE0,528,714,1538,1228,2020,0,0,0,0,...,32,60,168,152,276,2,0,0,0,24
2989,FF644E59F64C42AE852C8F5069733C22,1,446,726,106,1280,0,0,0,0,...,0,26,100,14,194,0,0,0,0,0
2990,FF761B75DB5D4869A8E264FB35E168F0,0,0,0,184,64,0,0,0,0,...,0,0,0,8,10,0,0,0,0,0


In [145]:
content_usage2=content_usage.groupby(['Subscription ID']).agg({"played date":"max"})
content_usage2=content_usage2.reset_index()
content_usage2.rename(columns={"played date" : "played_date_max"},inplace=True)

In [146]:
content_usage=pd.merge(content_usage1,content_usage2,on="Subscription ID",how="left")

In [147]:
content_usage.columns

Index(['Subscription ID', 'activity_asked_count', 'books_asked_count',
       'ela_lp_asked_count', 'math_learning_games_asked_count',
       'math_lp_asked_count', 'mathfacts_asked_count', 'others_asked_count',
       'playzone_asked_count', 'reading_asked_count', 'spelling_asked_count',
       'activity_time_spent_secs', 'books_time_spent_secs',
       'ela_lp_time_spent_secs', 'math_learning_games_time_spent_secs',
       'math_lp_time_spent_secs', 'mathfacts_time_spent_secs',
       'others_time_spent_secs', 'playzone_time_spent_secs',
       'reading_time_spent_secs', 'spelling_time_spent_secs',
       'activity_unique_playables_attempted',
       'books_unique_playables_attempted', 'ela_lp_unique_playables_attempted',
       'math_learning_games_unique_playables_attempted',
       'math_lp_unique_playables_attempted',
       'mathfacts_unique_playables_attempted',
       'others_unique_playables_attempted',
       'playzone_unique_playables_attempted',
       'reading_unique_play

In [148]:
# Merge the data sets
merged_data = pd.merge(user_attributes, email_data, on="Subscription ID", how="left")
merged_data = pd.merge(merged_data, content_usage, on="Subscription ID", how="left")

In [149]:
merged_data.columns

Index(['Subscription ID', 'cancellation date', 'plan type',
       'subscription date', 'student grade', 'acquisition type', 'is churned',
       'email_category_activation_count',
       'email_category_app_monthly_purchase_15day_count',
       'email_category_app_yearly_purchase_135day_count',
       ...
       'books_unique_playables_completed_attempted',
       'ela_lp_unique_playables_completed_attempted',
       'math_learning_games_unique_playables_completed_attempted',
       'math_lp_unique_playables_completed_attempted',
       'mathfacts_unique_playables_completed_attempted',
       'others_unique_playables_completed_attempted',
       'playzone_unique_playables_completed_attempted',
       'reading_unique_playables_completed_attempted',
       'spelling_unique_playables_completed_attempted', 'played_date_max'],
      dtype='object', length=121)

In [150]:
merged_data.head(4)

,Subscription ID,cancellation date,plan type,subscription date,student grade,acquisition type,is churned,email_category_activation_count,email_category_app_monthly_purchase_15day_count,email_category_app_yearly_purchase_135day_count,...,books_unique_playables_completed_attempted,ela_lp_unique_playables_completed_attempted,math_learning_games_unique_playables_completed_attempted,math_lp_unique_playables_completed_attempted,mathfacts_unique_playables_completed_attempted,others_unique_playables_completed_attempted,playzone_unique_playables_completed_attempted,reading_unique_playables_completed_attempted,spelling_unique_playables_completed_attempted,played_date_max
0,FFC9252D1BAB4F298A2179DCFB298FA2,NaT,yearly,2022-04-19,grade 2,paid,1,0,0,0,...,1.0,100.0,4.0,60.0,0.0,0.0,5.0,11.0,32.0,2023-05-12 20:42:09.830618
1,FF761B75DB5D4869A8E264FB35E168F0,NaT,yearly,2023-01-27,grade 1,paid,0,0,0,0,...,0.0,0.0,8.0,10.0,0.0,0.0,0.0,0.0,0.0,2023-01-27 03:04:53.309424
2,FF644E59F64C42AE852C8F5069733C22,2022-09-30,yearly,2022-01-20,kindergarten,organic,1,0,0,0,...,26.0,100.0,14.0,194.0,0.0,0.0,0.0,0.0,0.0,2023-04-05 22:09:21.000000
3,FF52DE445E1C4B3296B289C29EE2BAE0,NaT,family,2022-02-22,kindergarten,organic,1,0,0,0,...,60.0,168.0,152.0,276.0,2.0,0.0,0.0,0.0,24.0,2023-03-27 22:30:14.966679


In [151]:
merged_data.describe()

,is churned,email_category_activation_count,email_category_app_monthly_purchase_15day_count,email_category_app_yearly_purchase_135day_count,email_category_app_yearly_purchase_200day_count,email_category_app_yearly_purchase_260day_count,email_category_app_yearly_purchase_320day_count,email_category_app_yearly_purchase_75day_count,email_category_apps_day3_pa_1_count,email_category_apps_day3_pa_2_count,...,activity_unique_playables_completed_attempted,books_unique_playables_completed_attempted,ela_lp_unique_playables_completed_attempted,math_learning_games_unique_playables_completed_attempted,math_lp_unique_playables_completed_attempted,mathfacts_unique_playables_completed_attempted,others_unique_playables_completed_attempted,playzone_unique_playables_completed_attempted,reading_unique_playables_completed_attempted,spelling_unique_playables_completed_attempted
count,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,...,2992.000000,2992.000000,2992.000000,2992.000000,2992.000000,2992.000000,2992.0,2992.000000,2992.000000,2992.000000
mean,0.358509,0.125558,0.001275,0.000319,0.000319,0.000319,0.000319,0.000319,0.012110,0.007967,...,2.704211,8.333222,73.005013,38.098930,129.879345,10.768048,0.0,29.350936,5.536096,10.726939
std,0.479639,0.442603,0.035686,0.017851,0.017851,0.017851,0.017851,0.017851,0.115073,0.088915,...,18.141032,22.608265,203.796303,104.316656,332.873236,30.102128,0.0,106.818806,27.226627,36.134662
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.0,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,10.000000,4.000000,36.000000,0.000000,0.0,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,6.000000,60.000000,29.000000,118.000000,8.000000,0.0,10.000000,0.000000,6.000000
max,1.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,...,396.000000,300.000000,4698.000000,1683.000000,6028.000000,567.000000,0.0,2868.000000,606.000000,812.000000


In [152]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3138 entries, 0 to 3137
Columns: 121 entries, Subscription ID to played_date_max
dtypes: datetime64[ns](3), float64(40), int64(74), object(4)
memory usage: 2.9+ MB


In [153]:
merged_data.head(4)

,Subscription ID,cancellation date,plan type,subscription date,student grade,acquisition type,is churned,email_category_activation_count,email_category_app_monthly_purchase_15day_count,email_category_app_yearly_purchase_135day_count,...,books_unique_playables_completed_attempted,ela_lp_unique_playables_completed_attempted,math_learning_games_unique_playables_completed_attempted,math_lp_unique_playables_completed_attempted,mathfacts_unique_playables_completed_attempted,others_unique_playables_completed_attempted,playzone_unique_playables_completed_attempted,reading_unique_playables_completed_attempted,spelling_unique_playables_completed_attempted,played_date_max
0,FFC9252D1BAB4F298A2179DCFB298FA2,NaT,yearly,2022-04-19,grade 2,paid,1,0,0,0,...,1.0,100.0,4.0,60.0,0.0,0.0,5.0,11.0,32.0,2023-05-12 20:42:09.830618
1,FF761B75DB5D4869A8E264FB35E168F0,NaT,yearly,2023-01-27,grade 1,paid,0,0,0,0,...,0.0,0.0,8.0,10.0,0.0,0.0,0.0,0.0,0.0,2023-01-27 03:04:53.309424
2,FF644E59F64C42AE852C8F5069733C22,2022-09-30,yearly,2022-01-20,kindergarten,organic,1,0,0,0,...,26.0,100.0,14.0,194.0,0.0,0.0,0.0,0.0,0.0,2023-04-05 22:09:21.000000
3,FF52DE445E1C4B3296B289C29EE2BAE0,NaT,family,2022-02-22,kindergarten,organic,1,0,0,0,...,60.0,168.0,152.0,276.0,2.0,0.0,0.0,0.0,24.0,2023-03-27 22:30:14.966679


In [154]:
merged_data.isnull().sum()

Subscription ID                                     0
cancellation date                                2657
plan type                                           0
subscription date                                   0
student grade                                       0
                                                 ... 
others_unique_playables_completed_attempted       146
playzone_unique_playables_completed_attempted     146
reading_unique_playables_completed_attempted      146
spelling_unique_playables_completed_attempted     146
played_date_max                                   146
Length: 121, dtype: int64

In [156]:
datatime_columns = merged_data.select_dtypes(include="datetime64[ns]").columns

In [157]:
datatime_columns

Index(['cancellation date', 'subscription date', 'played_date_max'], dtype='object')

In [167]:
merged_data.columns

Index(['Subscription ID', 'cancellation date', 'plan type',
       'subscription date', 'student grade', 'acquisition type', 'is churned',
       'email_category_activation_count',
       'email_category_app_monthly_purchase_15day_count',
       'email_category_app_yearly_purchase_135day_count',
       ...
       'books_unique_playables_completed_attempted',
       'ela_lp_unique_playables_completed_attempted',
       'math_learning_games_unique_playables_completed_attempted',
       'math_lp_unique_playables_completed_attempted',
       'mathfacts_unique_playables_completed_attempted',
       'others_unique_playables_completed_attempted',
       'playzone_unique_playables_completed_attempted',
       'reading_unique_playables_completed_attempted',
       'spelling_unique_playables_completed_attempted', 'played_date_max'],
      dtype='object', length=121)

In [171]:
# def cdgrtrpd(cd,pdm):
#     if pd.notnull(cd) & pd.notnull(pdm):
#         cd=pd.to_datetime(cd)
#         pdm=pd.to_datetime(pdm)
#         return cd>pdm

In [175]:
# merged_data["label"]=merged_data.apply(lambda x : cdgrtrpd(x["cancellation date"],x["played_date_max"]),axis=1)

In [180]:
# merged_data.loc[merged_data.label==False]["label"][]

2       False
27      False
55      False
74      False
89      False
        ...  
3062    False
3083    False
3114    False
3125    False
3132    False
Name: label, Length: 204, dtype: object

In [181]:
merged_data.drop(columns=["label"],inplace=True)

In [182]:
merged_data.drop(columns=["cancellation date"],inplace=True)

In [183]:
merged_data.drop(columns=["played_date_max"],inplace=True)

In [185]:
merged_data["subscription date"]=pd.to_datetime(merged_data["subscription date"])

In [186]:
merged_data["subcription_month"]=merged_data["subscription date"].dt.month

In [187]:
merged_data.drop(columns=["subscription date"],inplace=True)

## EDA

In [188]:
merged_data.describe()

,is churned,email_category_activation_count,email_category_app_monthly_purchase_15day_count,email_category_app_yearly_purchase_135day_count,email_category_app_yearly_purchase_200day_count,email_category_app_yearly_purchase_260day_count,email_category_app_yearly_purchase_320day_count,email_category_app_yearly_purchase_75day_count,email_category_apps_day3_pa_1_count,email_category_apps_day3_pa_2_count,...,books_unique_playables_completed_attempted,ela_lp_unique_playables_completed_attempted,math_learning_games_unique_playables_completed_attempted,math_lp_unique_playables_completed_attempted,mathfacts_unique_playables_completed_attempted,others_unique_playables_completed_attempted,playzone_unique_playables_completed_attempted,reading_unique_playables_completed_attempted,spelling_unique_playables_completed_attempted,subcription_month
count,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,...,2992.000000,2992.000000,2992.000000,2992.000000,2992.000000,2992.0,2992.000000,2992.000000,2992.000000,3138.000000
mean,0.358509,0.125558,0.001275,0.000319,0.000319,0.000319,0.000319,0.000319,0.012110,0.007967,...,8.333222,73.005013,38.098930,129.879345,10.768048,0.0,29.350936,5.536096,10.726939,3.697897
std,0.479639,0.442603,0.035686,0.017851,0.017851,0.017851,0.017851,0.017851,0.115073,0.088915,...,22.608265,203.796303,104.316656,332.873236,30.102128,0.0,106.818806,27.226627,36.134662,3.032162
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,8.000000,0.000000,0.0,0.000000,0.000000,0.000000,2.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,10.000000,4.000000,36.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,6.000000,60.000000,29.000000,118.000000,8.000000,0.0,10.000000,0.000000,6.000000,4.000000
max,1.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,...,300.000000,4698.000000,1683.000000,6028.000000,567.000000,0.0,2868.000000,606.000000,812.000000,12.000000


In [189]:
merged_data.shape

(3138, 119)

## Missing Value Analysis

In [193]:
missing_value_Analysis=pd.DataFrame(merged_data.isnull().sum()/merged_data.shape[0])

In [196]:
missing_value_Analysis.reset_index(inplace=True)
missing_value_Analysis.rename(columns={"index":"Feature",0:"Missing_Value_%"})

,Feature,Missing_Value_%
0,Subscription ID,0.000000
1,plan type,0.000000
2,student grade,0.000000
3,acquisition type,0.000000
4,is churned,0.000000
...,...,...
114,others_unique_playables_completed_attempted,0.046526
115,playzone_unique_playables_completed_attempted,0.046526
116,reading_unique_playables_completed_attempted,0.046526
117,spelling_unique_playables_completed_attempted,0.046526


In [197]:
merged_data.dropna(inplace=True)

In [198]:
merged_data.shape

(2992, 119)

In [ ]:
merged_data.drop(columns=["Subscription ID"],inplace=True)

In [199]:
object_columns = merged_data.select_dtypes(include="object").columns

In [200]:
object_columns

Index(['Subscription ID', 'plan type', 'student grade', 'acquisition type'], dtype='object')

In [201]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2992 entries, 0 to 3137
Columns: 119 entries, Subscription ID to subcription_month
dtypes: float64(40), int64(75), object(4)
memory usage: 2.7+ MB


## Data Imbalance

In [282]:
merged_data["is churned"].value_counts()/merged_data.shape[0]

0    0.644719
1    0.355281
Name: is churned, dtype: float64

In [324]:
merged_data.isnull().sum()

plan type                                        0
student grade                                    0
acquisition type                                 0
is churned                                       0
email_category_activation_count                  0
                                                ..
others_unique_playables_completed_attempted      0
playzone_unique_playables_completed_attempted    0
reading_unique_playables_completed_attempted     0
spelling_unique_playables_completed_attempted    0
subcription_month                                0
Length: 118, dtype: int64

In [325]:
# merged_data.drop(columns=["Subscription ID"],inplace=True)

In [326]:
# Separate features (X) and target (y)
X = merged_data.drop("is churned", axis=1)
y = merged_data["is churned"]

In [327]:
# Convert column names to strings
X.columns = X.columns.astype(str)

In [328]:
object_columns = X.select_dtypes(include="object").columns
encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_encoded = pd.DataFrame(encoder.fit_transform(X[object_columns]))
X_encoded.columns = encoder.get_feature_names_out(object_columns)

/Users/satishkumarsahu/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [329]:
X_processed = pd.concat([X.drop(object_columns, axis=1), X_encoded], axis=1)

In [330]:
X_processed.columns

Index(['email_category_activation_count',
       'email_category_app_monthly_purchase_15day_count',
       'email_category_app_yearly_purchase_135day_count',
       'email_category_app_yearly_purchase_200day_count',
       'email_category_app_yearly_purchase_260day_count',
       'email_category_app_yearly_purchase_320day_count',
       'email_category_app_yearly_purchase_75day_count',
       'email_category_apps_day3_pa_1_count',
       'email_category_apps_day3_pa_2_count',
       'email_category_cancellation_confirmation_count',
       ...
       'plan type_yearly', 'student grade_grade 1', 'student grade_grade 2',
       'student grade_grade 3', 'student grade_grade 4',
       'student grade_grade 5', 'student grade_kindergarten',
       'student grade_pre kindergarten', 'acquisition type_organic',
       'acquisition type_paid'],
      dtype='object', length=127)

In [331]:
print("X_processed shape:", X_processed.shape)
print("y shape:", y.shape)

X_processed shape: (3132, 127)
y shape: (2992,)


In [332]:
X_processed=X_processed.iloc[0:2992]

In [333]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [334]:
X_train.fillna(0,inplace=True)
X_test.fillna(0,inplace=True)

In [335]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)
X_train
pca = PCA(n_components=0.95) # Retain 95% of the variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [336]:
dt_params = {
"max_depth": [None, 5, 10, 15],
"min_samples_split": [2, 5, 10],
"min_samples_leaf": [1, 2, 5]
}
dt_grid = GridSearchCV(DecisionTreeClassifier(), dt_params, cv=5)
dt_grid.fit(X_train_pca, y_train)
dt_best_model = dt_grid.best_estimator_

In [337]:
rf_params = {
"n_estimators": [100, 200, 300],
"max_depth": [None, 5, 10],
"min_samples_split": [2, 5, 10],
"min_samples_leaf": [1, 2, 5]
}
rf_grid = GridSearchCV(RandomForestClassifier(), rf_params, cv=5)
rf_grid.fit(X_train_pca, y_train)
rf_best_model = rf_grid.best_estimator_

In [338]:
xgb_params = {
"n_estimators": [100, 200, 300],
"max_depth": [3, 5, 7],
"learning_rate": [0.1, 0.01, 0.001]
}
xgb_grid = GridSearchCV(xgb.XGBClassifier(), xgb_params, cv=5)
xgb_grid.fit(X_train_pca, y_train)
xgb_best_model = xgb_grid.best_estimator_

In [340]:
from sklearn.inspection import permutation_importance

In [341]:
dt_permutation_importance = permutation_importance(dt_best_model, X_test_pca, y_test, n_repeats=10, random_state=42)

In [343]:
for feature, importance in zip(X_processed.columns, dt_permutation_importance.importances_mean):
    print(f"{feature}: {importance}")

email_category_activation_count: 0.04123539232053419
email_category_app_monthly_purchase_15day_count: 0.02520868113522534
email_category_app_yearly_purchase_135day_count: 0.0
email_category_app_yearly_purchase_200day_count: 0.0
email_category_app_yearly_purchase_260day_count: 0.0
email_category_app_yearly_purchase_320day_count: 0.0
email_category_app_yearly_purchase_75day_count: 0.0
email_category_apps_day3_pa_1_count: 0.0
email_category_apps_day3_pa_2_count: 0.0
email_category_cancellation_confirmation_count: 0.0005008347245408995
email_category_clevertap_count: 0.0
email_category_courses_d8_email_free_users_count: -0.00016694490818031094
email_category_deletionrequestmail_count: 0.0056761268781301945
email_category_inactive_for_7_days_count: 0.0
email_category_live_class_daily_reminder_count: 0.0
email_category_live_class_one_hour_reminder_count: 0.0
email_category_live_class_weekly_reminder_count: 0.0
email_category_live_class_welcome_email_count: 0.029382303839732858
email_category

In [345]:
# Evaluate the models
dt_accuracy = accuracy_score(y_test, dt_best_model.predict(X_test_pca))
rf_accuracy = accuracy_score(y_test, rf_best_model.predict(X_test_pca))
xgb_accuracy = accuracy_score(y_test, xgb_best_model.predict(X_test_pca))

print("Decision Tree Accuracy:", dt_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("XGBoost Accuracy:", xgb_accuracy)

Decision Tree Accuracy: 0.7045075125208681
Random Forest Accuracy: 0.7629382303839732
XGBoost Accuracy: 0.7879799666110183
